In [48]:
import speech_recognition as sr

recognizer = sr.Recognizer()
with sr.Microphone() as source:
    print("Listening...")
    recognizer.adjust_for_ambient_noise(source)  
    audio_data = recognizer.listen(source)
try:
    sentence = recognizer.recognize_google(audio_data)
    print("<<<<", sentence)
except sr.UnknownValueError:
    print("Sorry, could not understand audio.")
except sr.RequestError as e:
    print("Error fetching results from Google Speech Recognition service:", e)


Listening...
<<<< turn on the fans


In [49]:
from sklearn.metrics.pairwise import cosine_similarity; from transformers import AutoTokenizer, AutoModel; import torch; import numpy as np

sentences = ["turn on the fans", sentence]
model_name_or_path = "C:/MachineLearning/HuggingFace/HuggingFaceModels/bioasq-1m-msmarco-distilbert-gpl" 
tokenizer, model = AutoTokenizer.from_pretrained(model_name_or_path), AutoModel.from_pretrained(model_name_or_path)
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [50]:
with torch.no_grad():
    model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    model_outputs = model(**encoded_inputs)
embeddings = mean_pooling(model_outputs, encoded_inputs['attention_mask'])
similarities = cosine_similarity(embeddings[1:], embeddings[0].unsqueeze(0))

for i, similarity in enumerate(similarities, start=1):
    print(f"{sentences[i-1]:<25}: {similarity[0]}")
    
similarities[similarities < 0.6] = 0

turn on the fans         : 1.000000238418579


In [52]:
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Speed of speech (words per minute)
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)
if similarities > 1:
    engine.say("successful")
    engine.runAndWait()
else:
    engine.say("unsuccessful")
    engine.runAndWait()

In [45]:
from transformers import pipeline

classifier = pipeline(task="text-classification", model="C:\MachineLearning\HuggingFace\HuggingFaceModels\\roberta-base-go_emotions", top_k=None)
model_outputs = classifier(sentences)

flag = np.array([])
for output in model_outputs:
    sentence_flag = 1
    for item in output:
        if item['label'] == 'disapproval' and item['score'] > 0.01:
            sentence_flag = 0
            break  
    flag = np.append(flag, sentence_flag)

for i, f in enumerate(flag, start=1):
    print(f"{sentences[i-1]:<25} :    {f}")
flag = flag[1:]

turn on the fans          :    1.0
what the hell             :    0.0


In [46]:
result = np.multiply(flag.reshape(-1,1), similarities)
result[result > 0.7] = 1

import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # Speed of speech (words per minute)
engine.setProperty('volume', 1.0)  # Volume (0.0 to 1.0)
if result == 1:
    engine.say("successful")
    engine.runAndWait()
else:
    engine.say("unsuccessful")
    engine.runAndWait()
